# Example usage

To use `lisatools` in a project:

In [1]:
import lisatools

Standard library packages required to run this notebook:

In [2]:
import copy
import datetime

## The `Fund` and `Portfolio` classes

The `Fund` class stores general details about funds, such as their unique
ISIN identifier, the price, and the date the price was last updated.

In [3]:
ftse_global = lisatools.Fund(
    "FTSE Global All Cap Index Fund",
    172.14,
    isin="GB00BD3RZ582",
    date=datetime.date(2022, 11, 21)
)
print(ftse_global)

Fund('FTSE Global All Cap Index Fund', 172.14, date=datetime.date(2022, 11, 21), isin='GB00BD3RZ582')


An `ETF` subclass exists for exchange-traded funds, which additionally have a ticker symbol.

In [4]:
gilts = lisatools.ETF(
    "U.K. Gilt UCITS ETF",
    18.58,
    ticker="VGOV",
    isin="IE00B42WWV65",
    date=datetime.date(2022, 11, 21)
)
print(gilts)

ETF('U.K. Gilt UCITS ETF', 18.58, date=datetime.date(2022, 11, 21), isin='IE00B42WWV65', ticker='VGOV')


The `Portfolio` class represents the funds actually being held. Each line item
in the portfolio is represented as a `Holding`, and stores the fund details,
the number of units held, and the target allocation fraction.

In [5]:
holding1 = lisatools.Holding(ftse_global, 1.0, 0.6)
holding2 = lisatools.Holding(gilts, 5.0, 0.4)
pf = lisatools.Portfolio([holding1, holding2])
print(pf)

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   1.0000   172.14 0.6000 GB00BD3RZ582 2022-11-21
VGOV: U.K. Gilt UCITS ETF        5.0000    92.90 0.4000 IE00B42WWV65 2022-11-21


The total value of a holding is equal to the number of units held times the unit price.

In [6]:
f = lisatools.Fund(price = 2.0)
h = lisatools.Holding(f, units = 3.0)

h.value()

6.0

## Price updates using scraped FT data

So far, the pricing data has been provided manually. The `scraping` module provides tools to scrape the latest pricing data from the Financial Times' web site. This can be used to update all the prices in a `Portfolio` to the latest ones available:

In [7]:
pf.update_prices()
print(pf)

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   1.0000   171.69 0.6000 GB00BD3RZ582 2022-11-25
VGOV: U.K. Gilt UCITS ETF        5.0000    92.25 0.4000 IE00B42WWV65 2022-11-28


## Trading and the target portfolio

The example portfolio above requires some trades to be made in order to reach
the target portfolio as defined by the `target_fraction` attributes of its
`Holding`s.

This target portfolio can be extracted with the `target_portfolio()` method.

In [8]:
print(pf.target_portfolio())

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   0.9224   158.36 0.6000 GB00BD3RZ582 2022-11-25
VGOV: U.K. Gilt UCITS ETF        5.7223   105.58 0.4000 IE00B42WWV65 2022-11-28


The trades required to reach the target portfolio can be calculated using the
`trade_to_target()` method.

In [9]:
buy, sell = pf.trade_to_target()
print("Buy:\n=====", buy, "\nSell:\n=====", sell, sep = "\n")

Buy:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
VGOV: U.K. Gilt UCITS ETF        0.7223    13.33 0.4000 IE00B42WWV65 2022-11-28

Sell:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   0.0776    13.33 0.6000 GB00BD3RZ582 2022-11-25


We can generate buying instructions when we want to add new money to the 
portfolio by first making a dummy cash holding, and pretending to "sell" it.

To avoid making changes to the original portfolio, we must make a deep copy.

In [10]:
cash = lisatools.Fund("Cash")
cash_holding = lisatools.Holding(cash, units = 100)
new_pf = copy.deepcopy(pf)
new_pf.append(cash_holding)
print(new_pf)

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   1.0000   171.69 0.6000 GB00BD3RZ582 2022-11-25
VGOV: U.K. Gilt UCITS ETF        5.0000    92.25 0.4000 IE00B42WWV65 2022-11-28
Cash                           100.0000   100.00 0.0000 None         2022-11-28


Note that the dummy cash holding has its last-updated date set to today.

In [11]:
buy, sell = new_pf.trade_to_target()
print("Buy:\n=====", buy, "\nSell:\n=====", sell, sep = "\n")

Buy:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   0.2719    46.67 0.6000 GB00BD3RZ582 2022-11-25
VGOV: U.K. Gilt UCITS ETF        2.8903    53.33 0.4000 IE00B42WWV65 2022-11-28

Sell:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
Cash                           100.0000   100.00 0.0000 None         2022-11-28
